In [1]:
import geopandas
import pandas as pd
import numpy as np
import random
import math
from math import sqrt
import json
from shapely.geometry import Point

In [2]:
pesrons = pd.read_feather('data/trips_others.feather')

In [3]:
pesrons['home'] = pesrons.home.apply(lambda x: Point(x))
pesrons['coords'] = pesrons.coords.apply(lambda x: [Point(y) for y in x])

In [4]:
pesrons

,index,id,age,Ruta,Rutstorl,home,trip_chain,chain_length,home_ruta_storlek,coords
0,0,0,Ald7_15,3495006225750,250,POINT (12.577003233474832 56.15371189050657),[],0,3495006225750250,[POINT (12.577003233474832 56.15371189050657)]
1,1,1,Ald7_15,3495006225750,250,POINT (12.577639938001655 56.15307599069771),"[4, 4, 1]",3,3495006225750250,"[POINT (12.577639938001655 56.15307599069771),..."
2,2,2,Ald7_15,3495006225750,250,POINT (12.578197194684574 56.15334745858721),"[4, 4, 1]",3,3495006225750250,"[POINT (12.578197194684574 56.15334745858721),..."
3,3,3,Ald7_15,3495006225750,250,POINT (12.578197194684574 56.15334745858721),"[4, 1]",2,3495006225750250,"[POINT (12.578197194684574 56.15334745858721),..."
4,4,4,Ald7_15,3495006225750,250,POINT (12.579965306016872 56.15518771277077),"[4, 1, 16, 1]",4,3495006225750250,"[POINT (12.579965306016872 56.15518771277077),..."
...,...,...,...,...,...,...,...,...,...,...
1311072,1311072,1576774,Ald65_W,4320006189000,1000,POINT (13.917585218973997 55.85007009496898),"[7, 7, 12]",3,43200061890001000,"[POINT (13.917585218973997 55.85007009496898),..."
1311073,1311073,1576775,Ald65_W,4320006189000,1000,POINT (13.922698331888617 55.84516290224753),[],0,43200061890001000,[POINT (13.922698331888617 55.84516290224753)]
1311074,1311074,1576779,Ald7_15,4310006244000,1000,POINT (13.893047083591613 56.335926489142615),"[4, 1, 11, 1]",4,43100062440001000,[POINT (13.893047083591613 56.335926489142615)...
1311075,1311075,1576780,Ald7_15,4310006244000,1000,POINT (13.88484608563395 56.34245536086377),[],0,43100062440001000,[POINT (13.88484608563395 56.34245536086377)]


In [5]:
time_type = pd.read_feather('data/time-distr-type.feather')
time_type.columns = time_type.columns.astype(int)

In [6]:
time_type

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,3.363061e-07,6.516500e-08,1.031884e-08,4.488106e-08,2.236009e-07,3.606816e-10,8.176678e-10,4.742830e-10,7.004752e-09,4.703840e-12,1.843810e-14,2.865347e-07,4.736494e-08,2.170383e-10,1.041202e-17,0.000002,1.363705e-20
1,3.399703e-07,6.645439e-08,1.071433e-08,4.601146e-08,2.261108e-07,3.666990e-10,8.491035e-10,4.992388e-10,7.241303e-09,5.070375e-12,2.021318e-14,2.865271e-07,4.817936e-08,2.300867e-10,1.174859e-17,0.000002,1.558716e-20
2,3.436197e-07,6.776354e-08,1.112192e-08,4.715722e-08,2.285999e-07,3.727134e-10,8.815113e-10,5.253616e-10,7.483760e-09,5.464132e-12,2.215312e-14,2.864419e-07,4.899505e-08,2.438546e-10,1.325305e-17,0.000002,1.781123e-20
3,3.472532e-07,6.909258e-08,1.154184e-08,4.831806e-08,2.310670e-07,3.787214e-10,9.149110e-10,5.526976e-10,7.732185e-09,5.887027e-12,2.427263e-14,2.862792e-07,4.981162e-08,2.583775e-10,1.494601e-17,0.000002,2.034705e-20
4,3.508700e-07,7.044165e-08,1.197433e-08,4.949372e-08,2.335108e-07,3.847196e-10,9.493222e-10,5.812941e-10,7.986634e-09,6.341102e-12,2.658769e-14,2.860393e-07,5.062871e-08,2.736924e-10,1.685054e-17,0.000002,2.323750e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2.257721e-06,8.146123e-08,1.205123e-07,1.369419e-10,8.610269e-08,4.162963e-08,2.146314e-08,2.495146e-09,1.675800e-07,6.352232e-08,2.023622e-08,1.263815e-06,4.920724e-07,5.974976e-07,2.650646e-12,0.000001,2.142906e-09
1436,2.219677e-06,7.954619e-08,1.182902e-07,1.299678e-10,8.389298e-08,3.985303e-08,2.082373e-08,2.370975e-09,1.625417e-07,6.127276e-08,1.945581e-08,1.242614e-06,4.830201e-07,5.869131e-07,2.428372e-12,0.000001,2.006336e-09
1437,2.182006e-06,7.765913e-08,1.160905e-07,1.233163e-10,8.172561e-08,3.814403e-08,2.019810e-08,2.252487e-09,1.576155e-07,5.908974e-08,1.870165e-08,1.221575e-06,4.741123e-07,5.764295e-07,2.224155e-12,0.000001,1.877952e-09
1438,2.144709e-06,7.580018e-08,1.139135e-07,1.169741e-10,7.960012e-08,3.650042e-08,1.958615e-08,2.139447e-09,1.528005e-07,5.697177e-08,1.797299e-08,1.200700e-06,4.653455e-07,5.660474e-07,2.036580e-12,0.000001,1.757299e-09


In [7]:
def stop_coords(stop_id):
    lat = stops.loc[stop_id]['stop_lat']
    lon = stops.loc[stop_id]['stop_lon']
    latlon = (lat,lon)
    return latlon

In [8]:
class Population:
    def __init__(self):
        self.persons = []
    def to_JSON(self):
        return json.dumps(self, default=lambda o: _try(o), sort_keys=True, indent=4, separators=(',',':'))


class Person:
    def __init__(self):
        self.activities = []
        
#    def toJSON(self):
#        return json.dumps(self, default=lambda o: o.__dict__, 
#            sort_keys=True, indent=4)
        
#     def to_JSON(self):
#         return json.dumps(self, default=lambda o: _try(o), sort_keys=True, indent=4, separators=(',',':'))
        
def _try(o):
    try:
        if o.__class__ == ActivityType:
            raise Exception()
        return o.__dict__
    except:
        return str(o)

from enum import Enum
class ActivityType(Enum):
    HOME = 'HOME'
    WORK = 'WORK'

    def __str__(self):
        return self.name
    
class Activity(object):
    """Activity that is performed by a person.
    Person moves to a next activity after end_time is reached.

    Parameters
    ----------
    type_ : <ActivityType>
    coord : <Coord>
    start_time : <int> seconds from 00:00
    end_time : <int> seconds from 00:00
    """

    def __init__(self, type_, coord, start_time=None, end_time=None, zone=None):
        """docstring"""
        if start_time is None and end_time is None:
            raise Exception("Sanity check: both activity times are None")
        self.type = type_
        self.coord = coord
        self.start_time = start_time
        self.end_time = end_time
        self.zone = zone

    def __str__(self):
        return 'An ' + str(self.type) + ' at ' + str(self.coord)

In [9]:
def form_datetime(x):
    if type(x) in [np.float64, float, int, np.int64, np.int32]:
#         try:
#             x2 = math.trunc(x)
#         except TypeError:
#             x2 = x
        
        m = (x - math.trunc(x)) * 60
        s = math.trunc((m - math.trunc(m)) * 60)
        m = math.trunc(m)
        h = math.trunc(x)
        return datetime(year = 1970, month = 1 ,day = 1, hour = h, minute=m, second = s)
    elif type(x) == time:
        return datetime(year = 1970, month = 1 ,day = 1, hour = x.hour, minute=x.minute, second = x.second)
    
def form_td(x):
    if type(x) == np.float64:
        m = (x - math.trunc(x)) * 60
        s = math.trunc((m - math.trunc(m)) * 60)
        m = math.trunc(m)
        h = math.trunc(x)
        return td(hours = h, minutes=m, seconds = s)
    elif type(x) == time:
        return td(hours = x.hour, minutes=x.minute, seconds = x.second)

In [10]:
class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)
        
    def __str__(self):
        return str(self.lat) + ',' + str(self.lon)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon

    def __hash__(self):
        return hash((self.lat, self.lon))


In [11]:
kommun = geopandas.read_file('data/kommuns.geojson')
kommun = kommun.to_crs('EPSG:4326')

# persons to trips

In [12]:
pesrons.head()

,index,id,age,Ruta,Rutstorl,home,trip_chain,chain_length,home_ruta_storlek,coords
0,0,0,Ald7_15,3495006225750,250,POINT (12.577003233474832 56.15371189050657),[],0,3495006225750250,[POINT (12.577003233474832 56.15371189050657)]
1,1,1,Ald7_15,3495006225750,250,POINT (12.577639938001655 56.15307599069771),"[4, 4, 1]",3,3495006225750250,"[POINT (12.577639938001655 56.15307599069771),..."
2,2,2,Ald7_15,3495006225750,250,POINT (12.578197194684574 56.15334745858721),"[4, 4, 1]",3,3495006225750250,"[POINT (12.578197194684574 56.15334745858721),..."
3,3,3,Ald7_15,3495006225750,250,POINT (12.578197194684574 56.15334745858721),"[4, 1]",2,3495006225750250,"[POINT (12.578197194684574 56.15334745858721),..."
4,4,4,Ald7_15,3495006225750,250,POINT (12.579965306016872 56.15518771277077),"[4, 1, 16, 1]",4,3495006225750250,"[POINT (12.579965306016872 56.15518771277077),..."


In [17]:
pesrons = pesrons.sample(int(len(pesrons.index)/100))

In [18]:
pid = 0
population = Population()
for person in pesrons.itertuples():
    
    if len(person.trip_chain) == 0:
        continue
        
    for o, d, otype, dtype in zip(person.coords, person.coords[1:], [1] + person.trip_chain, person.trip_chain):

        trip = Person()
        trip.id = pid
        pid += 1
        trip.attributes = {'age': person.age}
        
        trip_start = time_type[dtype].sample(1, weights=time_type[dtype].values).index.values[0]*60

        act = Activity(otype,
                       Coord(latlon=[o.coords[0][1], o.coords[0][0]]),
                       start_time=0,
                       end_time=trip_start,
                       zone = int(kommun[kommun.contains(o)].kommun.values[0])
                      )
        trip.activities.append(act)
        
        act = Activity(dtype,
                       Coord(latlon=[d.coords[0][1], d.coords[0][0]]),
                       start_time=trip_start,
                       end_time=86400,
                       zone = int(kommun[kommun.contains(d)].kommun.values[0])
                      )
        trip.activities.append(act)
        population.persons.append(trip)

In [19]:
j = json.loads(population.to_JSON())

In [20]:
j

{'persons': [{'activities': [{'coord': {'lat': 55.72854687752723,
      'lon': 13.190576863340715},
     'end_time': '52920',
     'start_time': 0,
     'type': '7',
     'zone': 1281},
    {'coord': {'lat': 55.7038459777832, 'lon': 13.160027503967285},
     'end_time': 86400,
     'start_time': '52920',
     'type': '6',
     'zone': 1281}],
   'attributes': {'age': 'Ald45_64'},
   'id': 0},
  {'activities': [{'coord': {'lat': 55.7038459777832,
      'lon': 13.160027503967285},
     'end_time': '56460',
     'start_time': 0,
     'type': '2',
     'zone': 1281},
    {'coord': {'lat': 55.72854687752723, 'lon': 13.190576863340715},
     'end_time': 86400,
     'start_time': '56460',
     'type': '1',
     'zone': 1281}],
   'attributes': {'age': 'Ald45_64'},
   'id': 1},
  {'activities': [{'coord': {'lat': 55.4142330242821,
      'lon': 12.968638613167341},
     'end_time': '43620',
     'start_time': 0,
     'type': '5',
     'zone': 1233},
    {'coord': {'lat': 55.41415899833983, 'lon

In [22]:
with open('data/population_sjobo_others_sample.json', 'w') as outfile:
    json.dump(j, outfile)